In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

In [ ]:
#EJERCICIO 1 : Leerlo tratando de que Spark infiera el tipo de dato de cada columna, y cachearlo

In [ ]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
contractsDF = (spark.read.format("csv")\
        .option("header", "true")\
        .option("inferSchema", "true")\
        .option("quote", "\"")\
        .option("escape", "\"")\
        .option("mode", "DROPMALFORMED")\
        .load('C:/Users/laura.serrano/Desktop/Major_Contract_Awards.csv')).cache() 
contractsDF.printSchema()

In [ ]:
from pyspark.sql.types import DoubleType
assert(contractsDF.count() == 148515)

In [ ]:
#EJERCICIO 2

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
contractsDFenteros = (contractsDF.withColumn("Total Contract Amount (USD)",\
                                            F.regexp_replace("Total Contract Amount (USD)","\$","").cast(IntegerType()))).cache()

In [ ]:
contractsDFenteros.printSchema()

In [ ]:
from pyspark.sql.types import IntegerType
assert(len(contractsDFenteros.columns) == len(contractsDF.columns))
assert(contractsDFenteros.count() == contractsDF.count())
assert(contractsDFenteros.schema["Total Contract Amount (USD)"].dataType == IntegerType())

In [ ]:
#EJERCICIO 3

In [ ]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
contractsTranslatedDF = contractsDFenteros.withColumn("Region",\
                                                     F.when((F.col("Region") == "LATIN AMERICA AND CARIBBEAN"), "América Latina y Caribe")\
                                                    .when((F.col("Region") == "SOUTH ASIA"), "Asia del Sur")\
                                                    .when((F.col("Region") == "OTHER"), "Otro")\
                                                    .when((F.col("Region") == "AFRICA"), "África")\
                                                    .when((F.col("Region") == "MIDDLE EAST AND NORTH AFRICA"), "Oriente Medio y África del Norte")\
                                                    .when((F.col("Region") == "EAST ASIA AND PACIFIC"), "Asia Oriental y Pacífico")\
                                                    .when((F.col("Region") == "EUROPE AND CENTRAL ASIA"), "Europa y Asia Central"))

In [ ]:
assert(contractsTranslatedDF.select("Region").distinct().count() == 7)
g1 = contractsDF.groupBy("Region").count().withColumnRenamed("Region", "R1")
g2 = contractsTranslatedDF.groupBy("Region").agg(F.count("*").alias("c2"))
joinedDF = g1.join(g2, F.col("count") == g2.c2)
assert(joinedDF.count() == 7)
assert(joinedDF.where(F.col("count") == F.col("c2")).count() == 7)
assert(joinedDF.where(F.col("R1") == F.col("Region")).count() == 0)

In [ ]:
#EJERCICIO 4

In [ ]:
numeroCategoriasDF = contractsTranslatedDF.select(F.countDistinct(F.col("Procurement Type")).alias("Procurement Type"),\
                                                 F.countDistinct(F.col("Procurement Category")).alias("Procurement Category"),\
                                                 F.countDistinct(F.col("Procurement Method")).alias("Procurement Method"))
numeroCategoriasDF.show()

In [ ]:
assert(len(numeroCategoriasDF.columns) == 3)
assert(numeroCategoriasDF.count() == 1)
categorias = numeroCategoriasDF.collect()[0]
assert(categorias["Procurement Type"] == 60)
assert(categorias["Procurement Category"] == 5)
assert(categorias["Procurement Method"] == 18)

In [ ]:
#EJERCICIO 5 (NO HACER PIPELINE)

In [ ]:
#EJERCICIO 6

In [ ]:
from pyspark.sql import Window
paisWindow = Window().partitionBy("Borrower Country")
porcentajesDF = contractsDFenteros.withColumn("Total Pais",F.sum("Total Contract Amount (USD)").over(paisWindow))\
                                .withColumn("Porcentaje Pais", (F.col("Total Contract Amount (USD)"))*100/F.col("Total Pais"))\
                                .withColumn("Media Pais", F.avg("Total Contract Amount (USD)").over(paisWindow))\
                                .withColumn("Diff Porcentaje", ((F.col("Total Contract Amount (USD)")-F.col("Media Pais"))/\
                                                                        (F.col("Media Pais")))*100)



In [ ]:
assert("Total Pais" in porcentajesDF.columns)
assert("Porcentaje Pais" in porcentajesDF.columns)
r = porcentajesDF.where("`Project ID` = 'P069947'").head()
assert(r["Total Pais"] == 70485209)
assert(r["Porcentaje Pais"] - 25.985824912571374 < 0.001)
assert(r["Media Pais"] - 597332.279661017 < 0.001)
assert(r["Diff Porcentaje"] - 2966.3273396834217 < 0.001)